# Knowledge base System
문서를 이용한 벡터DB 구성 실습

In [ ]:
# 원본 tax.docx

# 문서 로딩
from langchain_community.document_loaders import Docx2txtLoader
loader = Docx2txtLoader('./tax_pure.docx')
docu = loader.load()

# 문서 분할
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, # 강사님이 추천해준 값
    chunk_overlap=200, # 강사님이 추천해준 값
)

doc_list = loader.load_and_split(text_splitter=splitter)

# 임베딩
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv

# 임베딩 모델
embedding = OpenAIEmbeddings(model='text-embedding-3-large')

# 벡터 DB 적재
from langchain_chroma import Chroma
database = Chroma.from_documents(
    documents=doc_list, 
    embedding=embedding,
    collection_name='chroma-tax',
    persist_directory='./chroma_origin' # 저장소 이름 다르게 해
)

# Retrieval
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
retrieved_docs = database.similarity_search(query, k=5)

# augumentation 
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o-mini')

# gpt-4o-mini
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
prompt = f"""[Identity]
- 당신은 최고의 한국 소득세 전문가 입니다.
- [context]만 참조해서 사용자의 질문에 답변해 주세요.
[context]
{retrieved_docs}
Questrion : {query}
"""
response = llm.invoke(prompt)
response

In [ ]:
# 표 마크타운으로 수정
# 문서 로딩
from langchain_community.document_loaders import Docx2txtLoader
loader = Docx2txtLoader('./tax.docx')
docu = loader.load()

# 문서 분할
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, # 강사님이 추천해준 값
    chunk_overlap=200, # 강사님이 추천해준 값
)

doc_list = loader.load_and_split(text_splitter=splitter)

# 임베딩
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv

# 임베딩 모델
embedding = OpenAIEmbeddings(model='text-embedding-3-large')

# 벡터 DB 적재
from langchain_chroma import Chroma
database = Chroma.from_documents(
    documents=doc_list, 
    embedding=embedding,
    collection_name='chroma-tax',
    persist_directory='./chroma_new' # 저장소 이름 다르게 해
)

# Retrieval
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
retrieved_docs = database.similarity_search(query, k=5)

# augumentation 
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o-mini')

# gpt-4o-mini

query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
prompt = f"""[Identity]
- 당신은 최고의 한국 소득세 전문가 입니다.
- [context]만 참조해서 사용자의 질문에 답변해 주세요.
[context]
{retrieved_docs}
Questrion : {query}
"""
response = llm.invoke(prompt)
response

AIMessage(content='연봉 5천만원인 거주자의 소득세를 계산하기 위해, 종합소득세 세율을 적용해야 합니다.\n\n1. **과세표준**: 5천만원 (5,000만 원)\n2. **적용되는 세율**: 5,000만 원 초과 ~ 8,800만 원 이하 구간에 해당하므로, 해당 구간의 세율을 적용합니다.\n\n   - 세율: 624만 원 + (5,000만 원 초과분 × 24%)\n   - 5,000만 원을 초과하는 금액: 5,000만 원\n\n계산해보면:\n\n- 초과분: 5,000만 원 - 5,000만 원 = 0 (추가 과세 없음)\n- 산출세액: 624만 원 + (0 × 24%) = 624만 원\n\n따라서 연봉 5천만원인 거주자의 소득세는 **624만 원**입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 219, 'prompt_tokens': 5078, 'total_tokens': 5297, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_8bda4d3a2c', 'id': 'chatcmpl-CDoB0ONerMlaiU4NAcR0DE1BfvTjs', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--0fa6bdb7-5522-4e06-bbae-d120d4316177-0', usage_metadata={'inpu